# Setup

In [1]:
# Check out the robocon2017 branch of some of our packages:
Pkg.checkout("RigidBodyDynamics", "robocon2017")
Pkg.checkout("RigidBodyTreeInspector", "robocon2017")
Pkg.checkout("DrakeVisualizer", "robocon2017")

INFO: Checking out RigidBodyDynamics robocon2017...
INFO: Pulling RigidBodyDynamics latest robocon2017...
INFO: No packages to install, update or remove
INFO: Checking out RigidBodyTreeInspector robocon2017...
INFO: Pulling RigidBodyTreeInspector latest robocon2017...
INFO: No packages to install, update or remove
INFO: Checking out DrakeVisualizer robocon2017...
INFO: Pulling DrakeVisualizer latest robocon2017...
INFO: No packages to install, update or remove


In [2]:
using RigidBodyDynamics
using StaticArrays
using Plots
using RigidBodyTreeInspector
using DrakeVisualizer

# Creating a double pendulum `Mechanism`

We're going to create a simple `Mechanism` that represents a [double pendulum](https://en.wikipedia.org/wiki/Double_pendulum). The `Mechanism` type can be thought of as an interconnection of rigid bodies and joints.

One way of creating a `Mechanism` is by parsing a [URDF](http://wiki.ros.org/urdf) file:

In [3]:
urdf = "doublependulum.urdf"
doublependulum = parse_urdf(Float64, urdf)

Vertex: world (root)
  Vertex: base_link, Edge: base_link_to_world
    Vertex: upper_link, Edge: shoulder
      Vertex: lower_link, Edge: elbow

Here are the bodies of our double pendulum:

In [4]:
collect(bodies(doublependulum))

4-element Array{RigidBodyDynamics.RigidBody{Float64},1}:
 RigidBody: "world"     
 RigidBody: "base_link" 
 RigidBody: "upper_link"
 RigidBody: "lower_link"

And here are its joints:

In [5]:
collect(joints(doublependulum))

3-element Array{RigidBodyDynamics.Joint{Float64},1}:
 Joint "base_link_to_world": Fixed joint                 
 Joint "shoulder": Revolute joint with axis [0.0,1.0,0.0]
 Joint "elbow": Revolute joint with axis [0.0,1.0,0.0]   

# The state of a `Mechanism`

A `Mechanism` stores the joint/rigid body layout, but no state information. State information is separated out into a `MechanismState` object:

In [6]:
const state = MechanismState(Float64, doublependulum)

MechanismState{Float64, Float64, Float64}(…)

Let's first set the configurations and velocities of the joints:

In [7]:
shoulder, elbow = (joints(doublependulum)...)
configuration(state, shoulder)[:] = 0.3
configuration(state, elbow)[:] = 0.4
velocity(state, shoulder)[:] = 1.
velocity(state, elbow)[:] = 2.;

**Important**: a `MechanismState` contains cache variables that depend on the configurations and velocities of the joints. These need to be invalidated when the configurations and velocities are changed. To do this, call

In [8]:
setdirty!(state)

The joint configurations and velocities are stored as `Vector`s (denoted $q$ and $v$ respectively in this package) inside the `MechanismState` object:

In [9]:
@show q = configuration_vector(state)
@show v = velocity_vector(state);

q = configuration_vector(state) = [0.4,0.0]
v = velocity_vector(state) = [2.0,0.0]


# Kinematics

We are now ready to do kinematics. Here's how you transform a point at the origin of the frame after the elbow joint to world frame:

In [10]:
world = root_body(doublependulum)
transform(state, Point3D(elbow.frameAfter, zeros(SVector{3})), default_frame(world))

Point3D in "world": [0.0,0.15,0.0]

Other objects like `Wrench`es, `Twist`s, and `SpatialInertia`s, etc. can be transformed in similar fashion.

You can also ask for the homogeneous transform to world:

In [11]:
transform_to_root(state, elbow.frameAfter)

Transform3D from "after_shoulder" to "world":
rotation: 0.3999999999999999 rad about [0.0,1.0,0.0], translation: [0.0,0.15,0.0]


Or a relative transform:

In [12]:
relative_transform(state, elbow.frameAfter, shoulder.frameAfter)

Transform3D from "after_shoulder" to "after_base_link_to_world":
rotation: 0.3999999999999999 rad about [0.0,1.0,0.0], translation: [0.0,0.15,0.0]


Here's the center of mass of the double pendulum:

In [13]:
center_of_mass(state)

Point3D in "world": [-0.486773,0.2,-1.15133]

# Dynamics

A `MechanismState` can also be used to compute quantities related to the dynamics of the `Mechanism`. Here we compute the mass matrix:

In [14]:
mass_matrix(state)

2×2 Symmetric{Float64,Array{Float64,2}}:
 4.663  2.33
 2.33   1.33

Here's the documentation for `mass_matrix` by the way:

In [15]:
?mass_matrix

search: mass_matrix mass_matrix!



```julia
mass_matrix(state)

```

Compute the joint-space mass matrix (also known as the inertia matrix) of the `Mechanism` in the given state, i.e., the matrix $M(q)$ in the unconstrained joint-space equations of motion

$$
M(q) \dot{v} + c(q, v, w_\text{ext}) = \tau
$$

This method implements the composite rigid body algorithm.


Note that there is also a zero-allocation version, `mass_matrix!` (the `!` at the end of a method is a Julia convention signifying that the function is 'in-place', i.e. modifies its input data).

We can do inverse dynamics as follows (note again that there is a non-allocating version of this method as well):

In [16]:
v̇ = [2.; 3.] # the joint acceleration vector, i.e., the time derivative of the joint velocity vector v
inverse_dynamics(state, v̇)

2-element Array{Float64,1}:
 25.8665
 12.4702

# Simulation

Let's simulate the double pendulum for 5 seconds, starting from the state we defined earlier. For this, we can use the basic `simulate` function:

In [17]:
ts, qs, vs = simulate(state, 5.);

`simulate` returns a vector of times (`ts`) and associated joint configurations (`qs`) and velocities (`vs`).

*Note:* A lower level interface for simulation/ODE integration with more options is also available. Consult the documentation for more information.

We can plot the trajectories using e.g. Plots.jl:

In [18]:
plot(ts, collect(q[1] for q in qs), xlabel = "Time [s]", ylabel = "Angle [rad]", lab = "Shoulder")

We can also show a 3D visualization of the double pendulum using DrakeVisualizer.

First, launch the viewer:

In [19]:
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window();

In [20]:
vis = Visualizer(parse_urdf(urdf, doublependulum));

In [21]:
# RigidBodyTreeInspector.animate(vis, doublependulum, ts, qs) # FIXME

# Symbolic dynamics

## Setup

In [22]:
# Make SymPy use version of Python downloaded through Julia's package system instead of system Python
ENV["PYTHON"]=""
Pkg.build("SymPy")

INFO: Building Conda
INFO: Building PyCall
INFO: Using the Python distribution in the Conda package by default.
To use a different Python version, set ENV["PYTHON"]="pythoncommand" and re-run Pkg.build("PyCall").


Fetching package metadata .........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/twan/code/julia/robocon2017/v0.5/Conda/deps/usr:
#
numpy                     1.12.0                   py27_0  


INFO: PyCall is using /Users/twan/code/julia/robocon2017/v0.5/Conda/deps/usr/bin/python (Python 2.7.12) at /Users/twan/code/julia/robocon2017/v0.5/Conda/deps/usr/bin/python, libpython = /Users/twan/code/julia/robocon2017/v0.5/Conda/deps/usr/lib/libpython2.7
INFO: /Users/twan/code/julia/robocon2017/v0.5/PyCall/deps/deps.jl has not changed
INFO: /Users/twan/code/julia/robocon2017/v0.5/PyCall/deps/PYTHON has not changed


In [23]:
using SymPy

In [24]:
# add a method to Base.flipsign and eps for SymPy. TODO: SymPy PR
Base.flipsign(sym::Sym, sign::Number) = sign > 0 ? sym : -sym;
Base.eps(::Type{SymPy.Sym}) = zero(Sym)

In [25]:
doublependulumsym = parse_urdf(Sym, urdf)
remove_fixed_joints!(doublependulumsym)

Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow

## Create symbolic parameters
* Masses: $m_1, m_2$
* Mass moments of inertia (about center of mass): $I_1, I_2$
* Link lengths: $l_1, l_2$
* Center of mass locations (w.r.t. preceding joint axis): $c_1, c_2$
* Gravitational acceleration: $g$

In [26]:
inertias = @syms m_1 m_2 I_1 I_2 positive = true
lengths = @syms l_1 l_2 c_1 c_2 real = true
gravitationalAcceleration = @syms g real = true
params = [inertias..., lengths..., gravitationalAcceleration...]

9-element Array{SymPy.Sym,1}
[m_1]
[   ]
[m_2]
[   ]
[I_1]
[   ]
[I_2]
[   ]
[l_1]
[   ]
[l_2]
[   ]
[c_1]
[   ]
[c_2]
[   ]
[ g ]

Unpack bodies and joints:

In [27]:
world, upperlink, lowerlink = (bodies(doublependulumsym)...)
shoulder, elbow = (joints(doublependulumsym)...);

Set symbolic parameters:

In [28]:
axis = shoulder.jointType.axis
spatial_inertia!(upperlink, SpatialInertia(shoulder.frameAfter, I_1 * axis * axis', SVector(0, 0, -c_1), m_1))
spatial_inertia!(lowerlink, SpatialInertia(elbow.frameAfter, I_2 * axis * axis', SVector(0, 0, -c_2), m_2))
add_frame!(upperlink, Transform3D(elbow.frameBefore, shoulder.frameAfter, SVector(0, 0, l_1)));

Create a symbolic `MechanismState`:

In [29]:
statesym = MechanismState(Sym, doublependulumsym);

Set the joint configuration and joint velocity vectors of the `MechanismState` to new vectors of symbolic variables:

In [30]:
configuration_vector(statesym)[:] = [symbols("q_$i", real = true) for i = 1 : num_positions(statesym)]

2-element Array{SymPy.Sym,1}
[q_1]
[   ]
[q_2]

In [31]:
velocity_vector(statesym)[:] = [symbols("v_$i", real = true) for i = 1 : num_positions(statesym)]

2-element Array{SymPy.Sym,1}
[v_1]
[   ]
[v_2]

Compute the mass matrix in symbolic form:

In [32]:
M = mass_matrix(statesym)
map!(simplify, M.data) # Note: M is a Symmetric matrix type; need to simplify the underlying data
full(M) # convert to full form so that it is pretty-printed (minor bug in SymPy.jl)

2×2 Array{SymPy.Sym,2}
[                                    2                            ]
[I_1 + I_2 - 2*c_2*l_1*cos(q_2) + l_1 *m_2  I_2 - c_2*l_1*cos(q_2)]
[                                                                 ]
[         I_2 - c_2*l_1*cos(q_2)                     I_2          ]

Kinetic energy:

In [33]:
simplify(kinetic_energy(statesym))

       2          2                        2                                  
I_1*v_1    I_2*v_1                  I_2*v_2               2                   
-------- + -------- + I_2*v_1*v_2 + -------- - c_2*l_1*v_1 *cos(q_2) - c_2*l_1
   2          2                        2                                      

                       2        2
                    l_1 *m_2*v_1 
*v_1*v_2*cos(q_2) + -------------
                          2      